In [1]:
# from SoccerNet.Downloader import SoccerNetDownloader as SNdl

In [2]:
# mySNdl = SNdl(LocalDirectory="data")
# mySNdl.downloadDataTask(task="mvfouls", split=["train","valid","test","challenge"], password="s0cc3rn3t")

In [1]:
from torchvision.io.video import read_video

In [2]:
video, audio, annot = read_video("data/mvfouls/challenge/action_0/clip_0.mp4")

/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [3]:
# Class name to label index

EVENT_DICTIONARY_action_class = {"Tackling":0,"Standing tackling":1,"High leg":2,"Holding":3,"Pushing":4,
                        "Elbowing":5, "Challenge":6, "Dive":7, "Dont know":8}

INVERSE_EVENT_DICTIONARY_action_class = {0:"Tackling", 1:"Standing tackling", 2:"High leg", 3:"Holding", 4:"Pushing",
                        5:"Elbowing", 6:"Challenge", 7:"Dive", 8:"Dont know"}


EVENT_DICTIONARY_offence_severity_class = {"No offence":0,"Offence + No card":1,"Offence + Yellow card":2,"Offence + Red card":3}

INVERSE_EVENT_DICTIONARY_offence_severity_class = {0:"No offence", 1:"Offence + No card", 2:"Offence + Yellow card", 3:"Offence + Red card"}


EVENT_DICTIONARY_offence_class = {"Offence":0,"Between":1,"No Offence":2, "No offence":2}

INVERSE_EVENT_DICTIONARY_offence_class = {0:"Offence", 1:"Between", 2:"No Offence"}


EVENT_DICTIONARY_severity_class = {"1.0":0,"2.0":1,"3.0":2,"4.0":3,"5.0":4}

INVERSE_EVENT_DICTIONARY_severity_class = {0:"No card", 1:"Borderline No/Yellow", 2:"Yellow card", 3:"Borderline Yellow/Red", 4:"Red card"}


EVENT_DICTIONARY_bodypart_class = {"Upper body":0,"Under body":1}

INVERSE_EVENT_DICTIONARY_bodypart_class = {0:"Upper body", 1:"Under body"}



EVENT_DICTIONARY = {'action_class':EVENT_DICTIONARY_action_class, 'offence_class': EVENT_DICTIONARY_offence_class, 
            'severity_class': EVENT_DICTIONARY_severity_class, 'bodypart_class': EVENT_DICTIONARY_bodypart_class, 'offence_severity_class': EVENT_DICTIONARY_offence_severity_class}
INVERSE_EVENT_DICTIONARY = {'action_class':INVERSE_EVENT_DICTIONARY_action_class, 'offence_class': INVERSE_EVENT_DICTIONARY_offence_class, 
            'severity_class': INVERSE_EVENT_DICTIONARY_severity_class, 'bodypart_class': INVERSE_EVENT_DICTIONARY_bodypart_class, 'offence_severity_class': INVERSE_EVENT_DICTIONARY_offence_severity_class}

In [4]:

import os
import torch
import json
# from config.classes import EVENT_DICTIONARY

##############################################################
#                                                            #
#              DO NOT MAKE ANY CHANGES HERE                  #
#                                                            #
##############################################################



# Function to load the labels from the json file
def label2vectormerge(folder_path, split, num_views):
	path_annotations = os.path.join(folder_path, split)
	path_annotations = os.path.join(path_annotations, "annotations.json") 

	dictionary_action = EVENT_DICTIONARY['action_class']

	if os.path.exists(path_annotations):
		with open(path_annotations) as f:
			train_annotations_data = json.load(f)
	else:
		print("PATH DOES NOT EXISTS")
		exit()

	not_taking = []

	num_classes_action = 8
	num_classes_offence_severity = 4

	labels_action = []
	labels_offence_severity= []
	number_of_actions = []

	total_distribution = torch.zeros(num_classes_offence_severity, num_classes_action)
	distribution_action = torch.zeros(1, num_classes_action)
	distribution_offence_severity = torch.zeros(1, num_classes_offence_severity)

	for actions in train_annotations_data['Actions']:
		action_class = train_annotations_data['Actions'][actions]['Action class']
		offence_class = train_annotations_data['Actions'][actions]['Offence']
		severity_class = train_annotations_data['Actions'][actions]['Severity']


		if action_class == '' or action_class == 'Dont know':
			not_taking.append(actions)
			continue

		if (offence_class == '' or offence_class == 'Between') and action_class != 'Dive':
			not_taking.append(actions)
			continue

		if (severity_class == '' or severity_class == '2.0' or severity_class == '4.0') and action_class != 'Dive' and offence_class != 'No offence' and offence_class != 'No Offence':
			not_taking.append(actions)
			continue

		if offence_class == '' or offence_class == 'Between':
			offence_class = 'Offence'

		if severity_class == '' or severity_class == '2.0' or severity_class == '4.0':
			severity_class = '1.0'

		if num_views == 1:
			for i in range(len(train_annotations_data['Actions'][actions]['Clips'])):
				if offence_class == 'No Offence' or offence_class == 'No offence':
					labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
					labels_offence_severity[len(labels_offence_severity)-1][0][0] = 1
					distribution_offence_severity[0][0] += 1
					off_index = 0
				elif offence_class == 'Offence' and severity_class == '1.0':
					labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
					labels_offence_severity[len(labels_offence_severity)-1][0][1] = 1
					distribution_offence_severity[0][1] += 1
					off_index = 1
				elif offence_class == 'Offence' and severity_class == '3.0':
					labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
					labels_offence_severity[len(labels_offence_severity)-1][0][2] = 1
					distribution_offence_severity[0][2] += 1
					off_index = 2
				elif offence_class == 'Offence' and severity_class == '5.0':
					labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
					labels_offence_severity[len(labels_offence_severity)-1][0][3] = 1
					distribution_offence_severity[0][3] += 1
					off_index = 3
				else:
					not_taking.append(actions)
					continue
				labels_action.append(torch.zeros(1, num_classes_action))
				labels_action[len(labels_action)-1][0][dictionary_action[action_class]] = 1
				distribution_action[0][dictionary_action[action_class]] += 1
				total_distribution[off_index][dictionary_action[action_class]] += 1
		else:
			if offence_class == 'No Offence' or offence_class == 'No offence':
				labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
				labels_offence_severity[len(labels_offence_severity)-1][0][0] = 1
				distribution_offence_severity[0][0] += 1
				index = 0
			elif offence_class == 'Offence' and severity_class == '1.0':
				labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
				labels_offence_severity[len(labels_offence_severity)-1][0][1] = 1
				distribution_offence_severity[0][1] += 1
				index = 1
			elif offence_class == 'Offence' and severity_class == '3.0':
				labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
				labels_offence_severity[len(labels_offence_severity)-1][0][2] = 1
				distribution_offence_severity[0][2] += 1
				index = 2
			elif offence_class == 'Offence' and severity_class == '5.0':
				labels_offence_severity.append(torch.zeros(1, num_classes_offence_severity))
				labels_offence_severity[len(labels_offence_severity)-1][0][3] = 1
				distribution_offence_severity[0][3] += 1
				index = 3
			else:
				not_taking.append(actions)
				continue

			number_of_actions.append(actions)
			labels_action.append(torch.zeros(1, num_classes_action))
			labels_action[len(labels_action)-1][0][dictionary_action[action_class]] = 1
			distribution_action[0][dictionary_action[action_class]] += 1
			total_distribution[index][dictionary_action[action_class]] += 1

	return labels_offence_severity, labels_action, distribution_offence_severity[0], distribution_action[0], not_taking, number_of_actions


# Function to load the path to the clips
def clips2vectormerge(folder_path, split, num_views, not_taking):

	path_clips = os.path.join(folder_path, split)

	if os.path.exists(path_clips):
		folders = 0

		for _, dirnames, _ in os.walk(path_clips):
			folders += len(dirnames) 
			
		clips = []
		for i in range(folders):
			if str(i) in not_taking:
				continue
			
			if num_views == 1:
				path_clip = os.path.join(path_clips, "action_" + str(i))
				path_clip_0 = os.path.join(path_clip, "clip_0.mp4")
				clips_all_view = []
				clips_all_view.append(path_clip_0)
				clips.append(clips_all_view)
				clips_all_view = []
				path_clip_1 = os.path.join(path_clip, "clip_1.mp4")
				clips_all_view.append(path_clip_1)
				clips.append(clips_all_view)
				clips_all_view = []

				if os.path.exists(os.path.join(path_clip, "clip_2.mp4")):
					path_clip_2 = os.path.join(path_clip, "clip_2.mp4")
					clips_all_view.append(path_clip_2)
					clips.append(clips_all_view)
					clips_all_view = []

				if os.path.exists(os.path.join(path_clip, "clip_3.mp4")):
					path_clip_3 = os.path.join(path_clip, "clip_3.mp4")
					clips_all_view.append(path_clip_3)
					clips.append(clips_all_view)
					clips_all_view = []
			else:
				path_clip = os.path.join(path_clips, "action_" + str(i))
				path_clip_0 = os.path.join(path_clip, "clip_0.mp4")
				clips_all_view = []
				clips_all_view.append(path_clip_0)
				path_clip_1 = os.path.join(path_clip, "clip_1.mp4")
				clips_all_view.append(path_clip_1)

				if os.path.exists(os.path.join(path_clip, "clip_2.mp4")):
					path_clip_2 = os.path.join(path_clip, "clip_2.mp4")
					clips_all_view.append(path_clip_2)

				if os.path.exists(os.path.join(path_clip, "clip_3.mp4")):
					path_clip_3 = os.path.join(path_clip, "clip_3.mp4")
					clips_all_view.append(path_clip_3)
				clips.append(clips_all_view)

		return clips




In [5]:
from torch.utils.data import Dataset
from random import random
import torch
import random
# from data_loader import label2vectormerge, clips2vectormerge
from torchvision.io.video import read_video


class MultiViewDataset(Dataset):
    def __init__(self, path, start, end, fps, split, num_views, transform=None, transform_model=None):

        if split != 'Chall':
            # To load the annotations
            self.labels_offence_severity, self.labels_action, self.distribution_offence_severity,self.distribution_action, not_taking, self.number_of_actions = label2vectormerge(path, split, num_views)
            self.clips = clips2vectormerge(path, split, num_views, not_taking)
            self.distribution_offence_severity = torch.div(self.distribution_offence_severity, len(self.labels_offence_severity))
            self.distribution_action = torch.div(self.distribution_action, len(self.labels_action))

            self.weights_offence_severity = torch.div(1, self.distribution_offence_severity)
            self.weights_action = torch.div(1, self.distribution_action)
        else:
            self.clips = clips2vectormerge(path, split, num_views, [])

        # INFORMATION ABOUT SELF.LABELS_OFFENCE_SEVERITY
        # self.labels_offence_severity => Tensor of size of the dataset. 
        # each element of self.labels_offence_severity is another tensor of size 4 (the number of classes) where the value is 1 if it is the class and 0 otherwise
        # for example if it is not an offence, then the tensor is [1, 0, 0, 0]. 

        # INFORMATION ABOUT SELF.LABELS_ACTION
        # self.labels_action => Tensor of size of the dataset. 
        # each element of self.labels_action is another tensor of size 8 (the number of classes) where the value is 1 if it is the class and 0 otherwise
        # for example if the action is a tackling, then the tensor is [1, 0, 0, 0, 0, 0, 0, 0]. 

        # INFORMATION ABOUT SLEF.CLIPS
        # self.clips => list of the size of the dataset
        # each element of the list is another list of size of the number of views. The list contains the paths to all the views of that particular action.

        # The offence_severity groundtruth of the i-th action in self.clips, is the i-th element in the self.labels_offence_severity tensor
        # The type of action groundtruth of the i-th action in self.clips, is the i-th element in the self.labels_action tensor
        
        self.split = split
        self.start = start
        self.end = end
        self.transform = transform
        self.transform_model = transform_model
        self.num_views = num_views

        self.factor = (end - start) / (((end - start) / 25) * fps)

        self.length = len(self.clips)
        print(self.length)

    def getDistribution(self):
        return self.distribution_offence_severity, self.distribution_action, 
    def getWeights(self):
        return self.weights_offence_severity, self.weights_action, 


    # RETURNS
    #
    # self.labels_offence_severity[index][0] => tensor of size 4. Example [1, 0, 0, 0] if the action is not an offence
    # self.labels_action[index][0] => tensor of size 8.           Example [1, 0, 0, 0, 0, 0, 0, 0] if the type of action is a tackling
    # videos => tensor of shape V, C, N, H, W with V = number of views, C = number of channels, N = the number of frames, H & W = height & width
    # self.number_of_actions[index] => the id of the action
    #
    def __getitem__(self, index):

        prev_views = []

        for num_view in range(len(self.clips[index])):

            index_view = num_view

            if len(prev_views) == 2:
                continue

            # As we use a batch size > 1 during training, we always randomly select two views even if we have more than two views.
            # As the batch size during validation and testing is 1, we can have 2, 3 or 4 views per action.
            cont = True
            if self.split == 'Train':
                while cont:
                    aux = random.randint(0,len(self.clips[index])-1)
                    if aux not in prev_views:
                        cont = False
                index_view = aux
                prev_views.append(index_view)


            video, _, _ = read_video(self.clips[index][index_view], output_format="THWC")
            frames = video[self.start:self.end,:,:,:]

            final_frames = None

            for j in range(len(frames)):
                if j%self.factor<1:
                    if final_frames == None:
                        final_frames = frames[j,:,:,:].unsqueeze(0)
                    else:
                        final_frames = torch.cat((final_frames, frames[j,:,:,:].unsqueeze(0)), 0)

            final_frames = final_frames.permute(0, 3, 1, 2)

            if self.transform != None:
                final_frames = self.transform(final_frames)

            final_frames = self.transform_model(final_frames)
            final_frames = final_frames.permute(1, 0, 2, 3)
            
            if num_view == 0:
                videos = final_frames.unsqueeze(0)
            else:
                final_frames = final_frames.unsqueeze(0)
                videos = torch.cat((videos, final_frames), 0)

        if self.num_views != 1 and self.num_views != 5:
            videos = videos.squeeze()   

        videos = videos.permute(0, 2, 1, 3, 4)

        if self.split != 'Chall':
            return self.labels_offence_severity[index][0], self.labels_action[index][0], videos, self.number_of_actions[index]
        else:
            return -1, -1, videos, str(index)

    def __len__(self):
        return self.length



In [29]:
import torchvision.transforms as transforms
from torchvision.models.video import R3D_18_Weights, MC3_18_Weights
from torchvision.models.video import R2Plus1D_18_Weights, S3D_Weights
from torchvision.models.video import MViT_V2_S_Weights, MViT_V1_B_Weights
from torchvision.models.video import mvit_v2_s, MViT_V2_S_Weights, mvit_v1_b, MViT_V1_B_Weights

path = "data/mvfouls"
start_frame = 0
end_frame = 125
fps = 25
num_views = 5
pre_model = "s3d"
max_num_worker = 8
batch_size = 2
data_aug = False

if data_aug == 'Yes':
        transformAug = transforms.Compose([
                                          transforms.RandomAffine(degrees=(0, 0), translate=(0.1, 0.1), scale=(0.9, 1)),
                                          transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
                                          transforms.RandomRotation(degrees=5),
                                          transforms.ColorJitter(brightness=0.5, saturation=0.5, contrast=0.5),
                                          transforms.RandomHorizontalFlip()
                                          ])
else:
    transformAug = None

if pre_model == "r3d_18":
    transforms_model = R3D_18_Weights.KINETICS400_V1.transforms()        
elif pre_model == "s3d":
    transforms_model = S3D_Weights.KINETICS400_V1.transforms()       
elif pre_model == "mc3_18":
    transforms_model = MC3_18_Weights.KINETICS400_V1.transforms()       
elif pre_model == "r2plus1d_18":
    transforms_model = R2Plus1D_18_Weights.KINETICS400_V1.transforms()
elif pre_model == "mvit_v2_s":
    transforms_model = MViT_V2_S_Weights.KINETICS400_V1.transforms()

dataset_Train = MultiViewDataset(path=path, start=start_frame, end=10, fps=fps, split='train',
            num_views = 2, transform=transformAug, transform_model=transforms_model)

2319


In [30]:
train_loader = torch.utils.data.DataLoader(dataset_Train,
            batch_size=batch_size, shuffle=True,
            num_workers=max_num_worker)

In [31]:
batch = dataset_Train[0]
print(len(batch))
print(batch[0])
print('-' * 100)
print(batch[1])
print('-' * 100)
print(batch[2].shape)
print('-' * 100)
print(batch[3])
print('-' * 100)

4
tensor([0., 1., 0., 0.])
----------------------------------------------------------------------------------------------------
tensor([0., 0., 0., 0., 0., 0., 1., 0.])
----------------------------------------------------------------------------------------------------
torch.Size([2, 3, 10, 224, 224])
----------------------------------------------------------------------------------------------------
0
----------------------------------------------------------------------------------------------------


In [32]:
def batch_tensor(tensor, dim=1, squeeze=False):
    """
    A function to reshape PyTorch tensor `tensor` along some dimension `dim` to the batch dimension 0 such that the tensor can be processed in parallel. 
    If `sqeeze`=True, the dimension `dim` will be removed completely, otherwise it will be of size=1. Check `unbatch_tensor()` for the reverese function.
    """
    batch_size, dim_size = tensor.shape[0], tensor.shape[dim]
    returned_size = list(tensor.shape)
    returned_size[0] = batch_size * dim_size
    returned_size[dim] = 1
    if squeeze:
        return tensor.transpose(0, dim).reshape(returned_size).squeeze_(dim)
    else:
        return tensor.transpose(0, dim).reshape(returned_size)


def unbatch_tensor(tensor, batch_size, dim=1, unsqueeze=False):
    """
    A function to chunk pytorch tensor `tensor` along the batch dimension 0 and concatenate the chuncks on dimension `dim` to recover from `batch_tensor()` function.
    If `unsqueee`=True, it will add a dimension `dim` before the unbatching. 
    """
    fake_batch_size = tensor.shape[0]
    nb_chunks = int(fake_batch_size / batch_size)
    if unsqueeze:
        return torch.cat(torch.chunk(tensor.unsqueeze_(dim), nb_chunks, dim=0), dim=dim).contiguous()
    else:
        return torch.cat(torch.chunk(tensor, nb_chunks, dim=0), dim=dim).contiguous()

In [33]:
from torch import nn

class WeightedAggregate(nn.Module):
    def __init__(self,  model, feat_dim, lifting_net=nn.Sequential()):
        super().__init__()
        self.model = model
        self.lifting_net = lifting_net
        num_heads = 8
        self.feature_dim = feat_dim

        r1 = -1
        r2 = 1
        self.attention_weights = nn.Parameter((r1 - r2) * torch.rand(feat_dim, feat_dim) + r2)

        self.normReLu = nn.Sequential(
            nn.LayerNorm(feat_dim),
            nn.ReLU()
        )        

        self.relu = nn.ReLU()
   


    def forward(self, mvimages):
        B, V, C, D, H, W = mvimages.shape # Batch, Views, Channel, Depth, Height, Width
        aux = self.lifting_net(unbatch_tensor(self.model(batch_tensor(mvimages, dim=1, squeeze=True)), B, dim=1, unsqueeze=True))


        ##################### VIEW ATTENTION #####################

        # S = source length 
        # N = batch size
        # E = embedding dimension
        # L = target length

        aux = torch.matmul(aux, self.attention_weights)
        # Dimension S, E for two views (2,512)

        # Dimension N, S, E
        aux_t = aux.permute(0, 2, 1)

        prod = torch.bmm(aux, aux_t)
        relu_res = self.relu(prod)
        
        aux_sum = torch.sum(torch.reshape(relu_res, (B, V*V)).T, dim=0).unsqueeze(0)
        final_attention_weights = torch.div(torch.reshape(relu_res, (B, V*V)).T, aux_sum.squeeze(0))
        final_attention_weights = final_attention_weights.T

        final_attention_weights = torch.reshape(final_attention_weights, (B, V, V))

        final_attention_weights = torch.sum(final_attention_weights, 1)

        output = torch.mul(aux.squeeze(), final_attention_weights.unsqueeze(-1))

        output = torch.sum(output, 1)

        return output.squeeze(), final_attention_weights


class ViewMaxAggregate(nn.Module):
    def __init__(self,  model, lifting_net=nn.Sequential()):
        super().__init__()
        self.model = model
        self.lifting_net = lifting_net

    def forward(self, mvimages):
        B, V, C, D, H, W = mvimages.shape # Batch, Views, Channel, Depth, Height, Width
        aux = self.lifting_net(unbatch_tensor(self.model(batch_tensor(mvimages, dim=1, squeeze=True)), B, dim=1, unsqueeze=True))
        pooled_view = torch.max(aux, dim=1)[0]
        return pooled_view.squeeze(), aux


class ViewAvgAggregate(nn.Module):
    def __init__(self,  model, lifting_net=nn.Sequential()):
        super().__init__()
        self.model = model
        self.lifting_net = lifting_net

    def forward(self, mvimages):
        B, V, C, D, H, W = mvimages.shape # Batch, Views, Channel, Depth, Height, Width
        aux = self.lifting_net(unbatch_tensor(self.model(batch_tensor(mvimages, dim=1, squeeze=True)), B, dim=1, unsqueeze=True))
        pooled_view = torch.mean(aux, dim=1)
        return pooled_view.squeeze(), aux


class MVAggregate(nn.Module):
    def __init__(self,  model, agr_type="max", feat_dim=400, lifting_net=nn.Sequential()):
        super().__init__()
        self.agr_type = agr_type

        self.inter = nn.Sequential(
            nn.LayerNorm(feat_dim),
            nn.Linear(feat_dim, feat_dim),
            nn.Linear(feat_dim, feat_dim),
        )

        self.fc_offence = nn.Sequential(
            nn.LayerNorm(feat_dim),
            nn.Linear(feat_dim, feat_dim),
            nn.Linear(feat_dim, 4)
        )


        self.fc_action = nn.Sequential(
            nn.LayerNorm(feat_dim),
            nn.Linear(feat_dim, feat_dim),
            nn.Linear(feat_dim, 8)
        )

        if self.agr_type == "max":
            self.aggregation_model = ViewMaxAggregate(model=model, lifting_net=lifting_net)
        elif self.agr_type == "mean":
            self.aggregation_model = ViewAvgAggregate(model=model, lifting_net=lifting_net)
        else:
            self.aggregation_model = WeightedAggregate(model=model, feat_dim=feat_dim, lifting_net=lifting_net)

    def forward(self, mvimages):

        pooled_view, attention = self.aggregation_model(mvimages)

        inter = self.inter(pooled_view)
        pred_action = self.fc_action(inter)
        pred_offence_severity = self.fc_offence(inter)

        return pred_offence_severity, pred_action, attention


In [34]:
import torch
from torchvision.models.video import r3d_18, R3D_18_Weights, MC3_18_Weights, mc3_18
from torchvision.models.video import r2plus1d_18, R2Plus1D_18_Weights, s3d, S3D_Weights
from torchvision.models.video import mvit_v2_s, MViT_V2_S_Weights, mvit_v1_b, MViT_V1_B_Weights



class MVNetwork(torch.nn.Module):

    def __init__(self, net_name='r2plus1d_18', agr_type='max', lifting_net=torch.nn.Sequential()):
        super().__init__()

        self.net_name = net_name
        self.agr_type = agr_type
        self.lifting_net = lifting_net
        
        self.feat_dim = 512

        if net_name == "r3d_18":
            weights_model = R3D_18_Weights.DEFAULT
            network = r3d_18(weights=weights_model)
        elif net_name == "s3d":
            weights_model = S3D_Weights.DEFAULT
            network = s3d(weights=weights_model)
            self.feat_dim = 400
        elif net_name == "mc3_18":
            weights_model = MC3_18_Weights.DEFAULT
            network = mc3_18(weights=weights_model)
        elif net_name == "r2plus1d_18":
            weights_model = R2Plus1D_18_Weights.DEFAULT
            network = r2plus1d_18(weights=weights_model)
        elif net_name == "mvit_v2_s":
            weights_model = MViT_V2_S_Weights.DEFAULT
            network = mvit_v2_s(weights=weights_model)
            self.feat_dim = 400
        else:
            weights_model = R2Plus1D_18_Weights.DEFAULT
            network = r2plus1d_18(weights=weights_model)
                
        network.fc = torch.nn.Sequential()

        self.mvnetwork = MVAggregate(
            model=network,
            agr_type=self.agr_type, 
            feat_dim=self.feat_dim, 
            lifting_net=self.lifting_net,
        )

    def forward(self, mvimages):
        return self.mvnetwork(mvimages)


In [35]:
model = MVNetwork(net_name='r2plus1d_18', agr_type='max')


In [36]:
LR = 0.001
weight_decay=0.001
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, 
                                    betas=(0.9, 0.999), eps=1e-07, 
                                    weight_decay=weight_decay, amsgrad=False)

In [37]:
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [38]:
batch_tensor

<function __main__.batch_tensor(tensor, dim=1, squeeze=False)>

In [40]:
weights_model = R2Plus1D_18_Weights.DEFAULT
network = r2plus1d_18(weights=weights_model).cuda()
for targets_offence_severity, targets_action, mvclips, action in dataset_Train:
    mvclips_cuda = mvclips.unsqueeze(0).cuda()
    print(mvclips_cuda.shape)
    continue
    batched_mvclips = batch_tensor(mvclips_cuda, dim=1, squeeze=True)
    enc_output = network(batched_mvclips)
    print(enc_output.shape)
    break

/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 3, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 4, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 3, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 3, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])
torch.Size([1, 3, 3, 10, 224, 224])
torch.Size([1, 2, 3, 10, 224, 224])


KeyboardInterrupt: 

In [45]:
print(list(network.parameters())[-3:])

[Parameter containing:
tensor([ 4.0572e-02,  3.7395e-02, -4.4805e-03,  4.4332e-02, -8.0134e-03,
         8.9692e-02,  4.4581e-02,  1.7988e-02,  7.9263e-02,  8.2089e-02,
         8.1651e-02,  1.2441e-01,  4.1011e-02,  5.2756e-02,  4.0515e-02,
         1.3831e-02,  4.9442e-02,  6.3419e-02,  6.5689e-03,  9.7634e-02,
        -4.9032e-02, -3.1900e-02,  1.5789e-02,  4.9427e-02,  3.3140e-02,
        -7.7997e-03,  5.6822e-02,  5.5623e-02,  7.0151e-02,  1.3241e-02,
         9.7916e-02,  9.8353e-03,  5.2859e-02,  2.0470e-02,  2.2956e-02,
         9.5345e-02,  1.0353e-02,  6.1648e-02,  6.0529e-02, -3.4429e-01,
         5.7699e-03,  5.6428e-02,  6.1417e-02,  2.3188e-02,  2.3180e-02,
         5.3957e-02,  9.3559e-02,  5.9049e-02,  2.6653e-02,  3.1645e-02,
         1.1577e-02,  2.2128e-02, -7.9027e-02,  5.8854e-02,  1.2945e-02,
        -8.2755e-03, -5.1337e-03,  7.8360e-02,  1.2926e-01,  6.0436e-02,
         4.0017e-02,  2.4657e-02,  2.3390e-02,  4.3700e-02,  4.0267e-02,
         6.1277e-03,  3.6907